Fitting a WL mass using `clmm`

_the LSST-DESC CLMM team_

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import clmm.polaraveraging as pa
import clmm.galaxycluster as gc
import clmm.modeling as modeling
import clmm
import sys
# This notebook must be run from the  
sys.path.append('./support')
import mock_data as mock
from numpy import random
from scipy import optimize as spo

In [ ]:
# define a true cosmology
# NB: need to cclify the astropy cosmology for generating the mock data (depends on modeling.py). 
# However, this is the astropy cosmology object that will need to be used on the data side, 
# profileaveraging.py (data side)

mock_cosmo = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)
# cclify allows access to the cosmo parameter the CCL way, but it is NOT a CCL cosmology object,
# but simply a dictionary
# mock_cosmo_ccl = clmm.cclify_astropy_cosmo(mock_cosmo) 

# Make mock data

In [ ]:
# define toy cluster parameters

cosmo = mock_cosmo
cluster_id = "Awesome_cluster"
cluster_m = 1.e15
cluster_z = 0.3
src_z = 0.8
concentration = 4
ngals = 10000
Delta = 200

In [ ]:
# make mock data object

ideal_data  = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration,
                                          cosmo, ngals, Delta, src_z)

In [ ]:
# make a clmm.GalaxyCluster object
# NB: mock data puts galaxy clusters in (0,0)

cluster_ra = 0.0
cluster_dec = 0.0
gc_object = clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec, 
                               cluster_z, ideal_data)

In [ ]:
# save the clmm.GalaxyCluster object

gc_object.save('mock_GC.pkl')

# Derive observables

In [ ]:
# load a clmm.GalaxyCluster object

cl = clmm.load_cluster('mock_GC.pkl')
print("Cluster info = ID:", cl.unique_id, "; ra:", cl.ra, "; dec:", cl.dec, "; z_l :", cl.z)
print ("The number of source galaxies is :", len(cl.galcat))

ra_l = cl.ra
dec_l = cl.dec
z = cl.z
e1 = cl.galcat['e1']
e2 = cl.galcat['e2']
ra_s = cl.galcat['ra']
dec_s = cl.galcat['dec']

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline

fsize = 15

fig = plt.figure(figsize=(10,6))
hb = fig.gca().hexbin(ra_s, dec_s, gridsize=50)

cb = fig.colorbar(hb)
cb.set_label('Number of sources in bin', fontsize=fsize)

plt.gca().set_xlabel(r'$\Delta RA$', fontsize=fsize)
plt.gca().set_ylabel(r'$\Delta Dec$', fontsize=fsize)
plt.gca().set_title('Source Galaxies', fontsize=fsize)

plt.show()

### Tangential shear, cross shear for each source galaxy in the cluster

In [ ]:
theta, g_t , g_x = pa.compute_shear(cl, geometry = "flat")

In [ ]:
fig = plt.figure(figsize=(10,6))

fig.gca().loglog(theta, g_t, '.')
plt.ylabel("reduced shear", fontsize=fsize)
plt.xlabel("angular distance [deg?]", fontsize=fsize)

### Make the binned profile

Using 2 different binnings to highlight the impact on the reconstructed mass when doing naive fitting (not accounting for the binning in the model estimation)

In [ ]:
# define bins
bin_edges1 = pa.make_bins(0.01, 3.7, 50)
bin_edges2 = pa.make_bins(0.01, 3.7, 10)

In [ ]:
fig = plt.figure(figsize=(10,6))

res1 = pa.make_shear_profile(cl, "radians","Mpc", bins=bin_edges1, cosmo = cosmo)
res2 = pa.make_shear_profile(cl, "radians","Mpc", bins=bin_edges2, cosmo = cosmo)

fig.gca().loglog(res1['radius'], res1['gt'], '.', label='50 bins')
fig.gca().loglog(res2['radius'], res2['gt'], '+', markersize=15, label='10 bins')
plt.legend(fontsize=fsize)
gt_profile1 = res1['gt']
r1 = res1['radius']

gt_profile2 = res2['gt']
r2 = res2['radius']

plt.gca().set_title(r'Binned shear of source galaxies', fontsize=fsize)
plt.gca().set_xlabel(r'$r\;(Mpc\;\;or\;\;h^{-1}\,Mpc?)$', fontsize=fsize)
plt.gca().set_ylabel(r'$g_t$', fontsize=fsize)

In [ ]:
# And now the galaxy cluster as a profile attribute
cl.profile

# Construct model

Simply estimating the model at the bin location. In that case, the mass reconstruction is dependent on the binning. Future developement would be to take the average of the model inside the bin instead, which should solve this issue for this set of ideal data.

In [ ]:
# select density profile parametrization and parameter values

# NB: the data are in physical Mpc, but modeling.py works in Mpc/h. So here, to build the model from bin positions, 
# we need to multiply the distance by h.

def nfw_to_shear_profile1(logm):
    m = 10.**logm
    gt_model = clmm.predict_reduced_tangential_shear(r1*cosmo.h, m, concentration, cluster_z, src_z, cosmo, 
                                                     Delta=200, halo_profile_parameterization='nfw')
    return sum((gt_model - gt_profile1) **2)

def nfw_to_shear_profile2(logm):
    m = 10.**logm
    gt_model = clmm.predict_reduced_tangential_shear(r2*cosmo.h, m, concentration, cluster_z, src_z, cosmo, 
                                                     Delta=200, halo_profile_parameterization='nfw')
    return sum((gt_model - gt_profile2) **2)

# Fit for mass

In [ ]:
# optimize to find the best-fit mass

logm_0 = random.uniform(13., 17., 1)[0]
logm_est1 = spo.minimize(nfw_to_shear_profile1, logm_0).x
logm_est2 = spo.minimize(nfw_to_shear_profile2, logm_0).x
m_est1 = 10.**logm_est1
m_est2 = 10.**logm_est2

In [ ]:
m_est1, m_est2

In [ ]:
import numpy as np
rr = np.logspace(-2,np.log10(5),100)

# NB: the data are in physical Mpc, but modeling.py works in Mpc/h. So here, to build the model from bin positions, 
# we need to multiply the distance by h.

gt_model1 = clmm.predict_reduced_tangential_shear(rr*cosmo.h, m_est1, concentration, cluster_z,src_z, cosmo,
                                                  Delta=200, halo_profile_parameterization='nfw')

gt_model2 = clmm.predict_reduced_tangential_shear(rr*cosmo.h, m_est2, concentration, cluster_z, src_z, cosmo,
                                                  Delta=200, halo_profile_parameterization='nfw')

In [ ]:
fig = plt.figure(figsize=(10,6))

fig.gca().scatter(r1, gt_profile1, label='mock data, M_input = %.3e Msun'%cluster_m)
fig.gca().plot(rr, gt_model1, label = 'best fit model, M_fit=%.3e'%m_est1, color='orange')
plt.semilogx()
plt.semilogy()

plt.legend()
plt.xlabel('R [Mpc]', fontsize=fsize)
plt.ylabel('reduced tangential shear', fontsize=fsize)

In [ ]:
fig = plt.figure(figsize=(10,6))

fig.gca().scatter(r2, gt_profile2, label='mock data, M_input = %.3e Msun'%cluster_m)
fig.gca().plot(rr, gt_model2, label = 'best fit model, M_fit=%.3e Msun'%m_est2, color='orange')
plt.semilogx()
plt.semilogy()

plt.legend(fontsize=fsize)
plt.xlabel('R [Mpc]', fontsize=fsize)
plt.ylabel('reduced tangential shear', fontsize=fsize)

# Cleanup some files used for the notebook

In [1]:
import os
os.remove('mock_GC.pkl')